# Projet Machine Learning : Xente Fraud Detection
Auteurs : Théo Engels, Hadrien Godts

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline as skPipeline
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import SMOTE, SMOTENC
from CustomTransformers import StringCleanTransformer, DayTimeTransformer, DropperTransformer, SignTransformer, OHTransformer, FloatTransformer, biningTransformer, weekdayTransformer
from sklearn.model_selection import train_test_split

from xgboost import XGBClassifier

from sklearn.model_selection import StratifiedKFold

In [3]:
# chargement des données
train = pd.read_csv("data/training.csv")
test = pd.read_csv("data/test.csv")

# shuffle et indication de la target
train = train.sample(frac=1).reset_index(drop=True)
train_Y = train.FraudResult
train.drop(['FraudResult'], axis=1, inplace=True)

# features dont le string est à nettoyer
StringToClean = ["TransactionId", "BatchId","AccountId","SubscriptionId","CustomerId", "ProviderId", "ProductId", "ChannelId", "ProductCategory"]

# features à dropper, one-hot-encoder et binariser respectivement, et initialisation du smote
drop_cols = ["CurrencyCode", "BatchId", "CountryCode", "CustomerId", "PricingStrategy", "Amount"]
hot_cols = ["ProductCategory"]
bin_cols = ["TransactionStartTime"]
smt  = SMOTE()

## Introduction
Ce rapport présente les processus et les résultats de notre travail sur le Xente fraud detection challenge. Nous commençons par une analyse des données présentes dans le dataset, suivie de la phase de feature engineering. Nous présentons ensuite les différents modèles que nous avons entrainés et leurs performances, et nous terminons par une courte conclusion et quelques perspectives de recherche.

L’objectif de ce challenge est de prédire si une transaction bancaire est frauduleuse ou pas, sur la base des différents features contenus dans le dataset.


## Analyse des données

La première chose importante à noter est que le training set contient 95 662 entrées, parmi lesquelles on compte 95 469 transactions légales et 193 frauduleuses (soit 0.2 %) : c’est un déséquilibre fort et important à noter, car le manque de transactions frauduleuses risque de compliquer l’entrainement des modèles. Nous pallions cette faiblesse dans le feature engineering.

In [4]:
# Code pour nbr transactions
print("Nb de transactions : ",train_Y.count())
print("Nb de fraudes : ",train_Y.value_counts()[1])
print("Nb de non fraudes : ",train_Y.value_counts()[0])
print("ratio fraudes/non frondes : ",train_Y.value_counts()[1] / train_Y.value_counts()[0])


Nb de transactions :  95662
Nb de fraudes :  193
Nb de non fraudes :  95469
ratio fraudes/non frondes :  0.0020215986341115964


La seconde chose importante à propos des données est qu’il ne manque aucune donnée : toutes les entrées ont des données pour chaque feature.

In [5]:
# Code pour données manquantes

print("Are there missing values : ", train.isnull().values.any())

train.head()

Are there missing values :  False


,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy
0,TransactionId_72536,BatchId_94324,AccountId_3432,SubscriptionId_841,CustomerId_3867,UGX,256,ProviderId_6,ProductId_3,airtime,ChannelId_3,1000.0,1000,2019-02-04T19:23:42Z,2
1,TransactionId_51068,BatchId_117017,AccountId_1653,SubscriptionId_443,CustomerId_2033,UGX,256,ProviderId_1,ProductId_15,financial_services,ChannelId_3,5000.0,5000,2019-01-16T14:44:23Z,2
2,TransactionId_18503,BatchId_57323,AccountId_4584,SubscriptionId_4030,CustomerId_5048,UGX,256,ProviderId_6,ProductId_3,airtime,ChannelId_3,2000.0,2000,2019-02-10T10:48:40Z,2
3,TransactionId_140415,BatchId_43725,AccountId_4841,SubscriptionId_3829,CustomerId_800,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-5000.0,5000,2019-02-08T15:18:54Z,2
4,TransactionId_4562,BatchId_25375,AccountId_4249,SubscriptionId_4429,CustomerId_7343,UGX,256,ProviderId_4,ProductId_3,airtime,ChannelId_2,-30000.0,30000,2018-12-04T15:51:44Z,4


Voyons ensuite les différents feature du dataset. Nous mentionnons ici les détails importants ainsi que les premières décisions prises concernant certaines features.
-	TransactionId : numéro de la transaction. Cette donnée est à garder, car elle permet de soumettre les résultats au défi, mais nous retirerons cette colonne du dataset dans la mesure où le numéro de la transaction ne devrait rien nous apprendre de particulier sur son caractère frauduleux.

-	BatchId : le numéro de groupe de transaction. Comme pour le numéro de transaction, cette donnée ne devrait rien nous apprendre sur le caractère frauduleux d’une transaction.

-	AccountId : l’identifiant d’un compte client. Nous conservons cette donnée.

-	SubscriptionId : l’identifiant d’une souscription. D’après la description des features, cette donnée semble liée à un compte client, sans autre explication particulièrement révélatrice. Vu ce lien et ce manque de clarté, nous décidons de laisser cette variable hors du dataset, afin d’en réduire la taille.
-	CustomerId : l’identifiant d’un client. À priori, un client peut disposer de plusieurs comptes, donc pour réduire la taille du dataset, nous laissons cette variable hors du dataset en privilégiant AccountId. Cela étant, si on peut établir qu’un compte fraude plus que les autres, il sera pertinent de reconstruire le lien vers le CustomerId.
-	CurrencyCode : la monnaie de la transaction. Il n’y a qu’une seule valeur pour ce feature, donc nous le laissons également hors du dataset.
-	CountryCode : le code du pays de la transaction. Idem que CurrencyCode, nous le laissons donc également hors du dataset.
-	ProviderId : indique la source du produit acheté via la transaction. Dans un premier temps, nous conservons cette feature en faisant un one-hot encoding sur les six valeurs possibles, en partant du principe qu'il peut y avoir un lien entre la source du produit et le caractère frauduleux de la transaction.
-	ProductId : donne le code du produit qui a été acheté lors de la transaction. Cette colonne contient 23 valeurs possibles, pour éviter de multiplier les features, nous  ne faisons dans un premier temps pas de oneHot encoding sur celle-ci. En outre, nous disposons de la variable ProductCategory (voir ci-après) donc nous décidons de laisser celle-ci hors du dataset. 
-	ProductCategory : indique la catégorie de produit qui a été fait l’objet de la transaction. Il y a 9 catégories possibles, donc nous pouvons faire du one-hot encoding pour conserver cette feature.
-	ChannelId : indique la méthode de paiement utilisée pour effectuer la transaction. Cette feature nous semble importante, nous la conservons donc en faisant du one-hot encoding une fois de plus.
-	Amount : indique le montant de la transaction, avec son signe (+ ou - pour crédit ou débit). La feature value nous donne déjà le montant absolu, nous faisons donc d'abord du binary encoding pour que les montants positifs soient désormais true, et les montants négatifs false.
-	Value : indique le montant de la transaction, sans son signe. Nous conservons cette feature.
-	TransactionStartTime : indique la date et l'horaire de la transaction. Nous convertissons le format de cette donnée pour avoir un ordre chronologique, puis nous la séparons en une colonne date et une colonne heure. La colonne date est convertie pour avoir le jour de la semaine. La colonne heure laissée telle quelle pour le moment.
-	PricingStrategy : indique la structure de prix de Xente pour ses différents marchands. Sans certitude sur ce que cette colonne représente exactement, nous la one-hot encodons et verrons par la suite si elle a un impact ou non.
-	FraudResult : indique si la transaction est frauduleuse. Il s’agit de notre target, nous la gardons donc.

In [6]:
# Code pour nunique

print("nunique :\n", train.nunique())

nunique :
 TransactionId           95662
BatchId                 94809
AccountId                3633
SubscriptionId           3627
CustomerId               3742
CurrencyCode                1
CountryCode                 1
ProviderId                  6
ProductId                  23
ProductCategory             9
ChannelId                   4
Amount                   1676
Value                    1517
TransactionStartTime    94556
PricingStrategy             4
dtype: int64


## Feature engineering

Le dataset est déjà séparé en training et en validation, ce qui nous épargne cette étape, et nous l'avons déjà mélangé au moment de la lecture. Afin d’organiser efficacement notre travail, nous rédigeons le code sous forme de pipelines.

Nous créons un préprocesseur pour la transformation des données que nous appliquons au training set. Les transformations du premier préprocesseur sont décrites dans le tableau suivant :

| Transformateur | Description |
| --- | --- |
| clean | Pour les colonnes dont les données sont précédées d'un string (par exemple : BatchID578), retire le texte et ne conserve que les données numériques |
| amount_to_sign | Remplace les valeurs de la colonne amount par true si elles sont positives et false si elles sont négatives ; la colonne est renommée « sign ». |
| day_time_separator | Convertis le TransactionStartTime en format yyyymmddhhmmss, et sépare la colonne en jour (TransactionStartDay) et en heure (TransactionStartTime). |
| dropper | Supprime les features non-pertinentes, qui sont listées dans la liste "drop-cols"|
| one_hot_encode | Effectue un one-hot-encoding pour les colonnes de la list "hot_cols"|
| weekday | Convertis la colonne TransactionStartDay en jour de la semaine (représenté par un entier allant de 1 à 7) plutôt qu'en date  |
| float | Impose le type float sur toutes les colonnes (certaines colonnes du one-hot encoding ne sont plus en float). De plus, il passe les transactionId (dont nous avons besoin pour la soumission) en index. |
| smote | Applique un suréchantillonnage artificiel sur le dataset afin d'équilibrer le dataset. |

Notez que le smote sera ignoré lors du predict(test)	

In [7]:
# Code pour le pipeline

pipeline = imbPipeline(steps = [
    ("clean", StringCleanTransformer()),
    ("amout to sign", SignTransformer()),
    ("day_time_separator", DayTimeTransformer()),
    ("Dropper", DropperTransformer(drop_cols)),
    ("One hot encoding", OHTransformer(hot_cols)),
    ("weekday", weekdayTransformer()),
    ("float", FloatTransformer()),
    ("smote", smt),
    ("model", XGBClassifier(n_estimators = 500))
])

## Mesure de performances
Afin d'estimer la performance de notre modèle, il est possible de séparer notre dataset en un set d'entrainement et un set de validation. Le set de validation permet de mesurer le score "f1" du modèle, cependant le score calculé par cette méthode est bien plus élevé que le score calculé sur le site du challenge. Ce problème peut potentiellement venir d'une fuite de données lors de notre feature engineering, cependant la source exacte n'a pas été trouvée.
Cette méthode d'évaluation a donc été abandonnée.

| Modèle | F1 score crossvalidation | Public Score | Private Score |
|---|---|---|---|
| XGB | 0.8984 | 0.730158730 | 0.738461538 |



In [8]:
def OH(X, hot_cols):
    OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
    for elem in hot_cols:
        OH_cols = pd.DataFrame(OH_encoder.fit_transform(X[elem].values.reshape(-1,1)))
        OH_cols.rename(columns=lambda x: elem + str(x), inplace=True)
        OH_cols.index = X.index
        X = pd.concat([X, OH_cols], axis=1)
    return X

drop_cols_diff = ["CurrencyCode", "BatchId", "CountryCode", "CustomerId", "PricingStrategy", "Amount", "ProductCategory"]

pipelineXGB_diff= imbPipeline(steps = [
    ("clean", StringCleanTransformer()),
    ("amout to sign", SignTransformer()),
    ("day_time_separator", DayTimeTransformer()),
    ("Dropper", DropperTransformer(drop_cols_diff)),
    # ("One hot encoding", OHTransformer(hot_cols)),
    ("weekday", weekdayTransformer()),
    ("float", FloatTransformer()),
    ("smote", smt),
    ("model", XGBClassifier())
])

if (True):

    print("cross validation score : ", cross_val_score(pipelineXGB_diff, OH(train.copy(), hot_cols), train_Y, cv=5, scoring="f1").mean())

    pipelineXGB_diff.fit(OH(train.copy(), hot_cols), train_Y)
    test_res = pipelineXGB_diff.predict(OH(test.copy(), hot_cols))

    output = pd.DataFrame()
    output["TransactionId"] = test["TransactionId"]
    output["FraudResult"] = test_res

    #save the result to csv file
    output.to_csv("submission"+"diff_CV_zindi"+".csv", index=False)


cross validation score :  0.8984212076215782


## Modèles
Nous entrainons les données adaptées sur trois modèles : un Random Forest, un XGBoost et un MLP ; le tableau suivant montre les performances de ces modèles, sans adaptation particulière de leurs paramètres.


In [9]:
# Code pour le modèle de base

drop_cols = ["CurrencyCode", "BatchId", "CountryCode", "CustomerId", "PricingStrategy", "Amount"]
hot_cols = ["ProductCategory"]


pipelineRF = imbPipeline(steps = [
    ("clean", StringCleanTransformer()),
    ("amout to sign", SignTransformer()),
    ("day_time_separator", DayTimeTransformer()),
    ("Dropper", DropperTransformer(drop_cols)),
    ("One hot encoding", OHTransformer(hot_cols)),
    ("weekday", weekdayTransformer()),
    ("float", FloatTransformer()),
    ("smote", smt),
    ("model", RandomForestClassifier())
])

pipelineXGB= imbPipeline(steps = [
    ("clean", StringCleanTransformer()),
    ("amout to sign", SignTransformer()),
    ("day_time_separator", DayTimeTransformer()),
    ("Dropper", DropperTransformer(drop_cols)),
    ("One hot encoding", OHTransformer(hot_cols)),
    ("weekday", weekdayTransformer()),
    ("float", FloatTransformer()),
    ("smote", smt),
    ("model", XGBClassifier())
])

pipelineMLP= imbPipeline(steps = [
    ("clean", StringCleanTransformer()),
    ("amout to sign", SignTransformer()),
    ("day_time_separator", DayTimeTransformer()),
    ("Dropper", DropperTransformer(drop_cols)),
    ("One hot encoding", OHTransformer(hot_cols)),
    ("weekday", weekdayTransformer()),
    ("float", FloatTransformer()),
    ("smote", smt),
    ("model", MLPClassifier())
])

if (False):
    for pipeline, name in zip([pipelineRF, pipelineXGB, pipelineMLP],["pipelineRF", "pipelineXGB", "pipelineMLP"]):
        # print("cross validation score : ", cross_val_score(pipeline, train, train_Y, cv=5, scoring="f1").mean())
        copy_train = train.copy()
        copy_train_Y = train_Y.copy()
        pipeline.fit(copy_train, copy_train_Y)
        test_res = pipeline.predict(test.copy())

        output = pd.DataFrame()
        output["TransactionId"] = test["TransactionId"]
        output["FraudResult"] = test_res

        #save the result to csv file
        output.to_csv("submission"+name+".csv", index=False)




Les résultats sont les suivants :

| Modèle | Public score | Private score |
| --- | --- | --- |
| Random Forest | 0.561403508 | 0.517241379 |
| XGBoost | 0.741935483 | 0.744186046 |
| MLP | 0.070323488 | 0.070412999 |

![Random Forest Result](./RFinitialScore.png)
![XGBoost Result](./XGBoostinitialScore.png)
![MLP Result](./MLPinitialScore.png)

Nous voyons clairement que le XGBoost présente les meilleurs résultats. Nous nous tentons donc de partir de cette base pour améliorer encore le modèle. Nous travaillons principalement sur les hyperparamètres suivants  :
-	n_estimators
-	learning rate
-	max_depth



## Optimisation des hyperparamètres

L'optimisation des hyperparamètres des modèles permettrait d'améliorer les performances de ceux-ci. Nous avons utilisé la fonction "GridSearchCV" qui prend en paramètres un dictionnaire contenant plusieurs valeurs pour les paramètres du pipeline et qui sélectionne la meilleure combinaison parmi toutes celles possibles. Ce code a été lancé plusieurs fois pour chacun des modèles en utilisant a chaque fois de paramètres de plus en plus précis afin de trouver les meilleurs paramètres possibles (run 1 :n-estimators = [50,100,500,700],... run 2 :n-estimators = [400,500,600],... run 3 :n-estimators = [550,600,650],...).

Le fonctionnement de cette fonction nous oblige a sortir le oneHot encoding du pipeline, car elle sépare elle-même le dataset pour faire de la crossvalidation, cela peut mener à des problèmes ou un subSet de donnée ne contient pas le même nombre de colonnes, car une de valeurs possible pour une colonne oneHot encodées n'est pas présente dans celui-ci. Encodée tout le data set avant de l'envoyé dans le pipeline permet donc d'éviter ce problème.

Les modèles utilisant les paramètres optimisés donnent de moins bons scores privé et public sur le site du challenge, cela vient de l'estimation de la performance du modèle qui n'est pas correct, et qui empêche donc de converger vers des paramètres corrects.

| Modèle | Public Score | Private Score |
|---|---|---|
| XGB opti | 0.700000000 | 0.704000000 |
| XGB non-opti | 0.754098360 | 0.749999999 |


Nous avons donc décidé de garder le modèle non optimisé, en sachant qu'il est possible de l'améliorer avec de meilleurs hyperparamètres. 

In [10]:
def OH(X, hot_cols):
    OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
    for elem in hot_cols:
        OH_cols = pd.DataFrame(OH_encoder.fit_transform(X[elem].values.reshape(-1,1)))
        OH_cols.rename(columns=lambda x: elem + str(x), inplace=True)
        OH_cols.index = X.index
        X = pd.concat([X, OH_cols], axis=1)
    return X

drop_cols_opti = ["CurrencyCode", "BatchId", "CountryCode", "CustomerId", "PricingStrategy", "Amount", "ProductCategory"]

pipelineXGB_opti= imbPipeline(steps = [
    ("clean", StringCleanTransformer()),
    ("amout to sign", SignTransformer()),
    ("day_time_separator", DayTimeTransformer()),
    ("Dropper", DropperTransformer(drop_cols_opti)),
    # ("One hot encoding", OHTransformer(hot_cols)),
    ("weekday", weekdayTransformer()),
    ("float", FloatTransformer()),
    ("smote", smt),
    ("model", XGBClassifier())
])


#selection des paramètres par itération manuelle de ce bout de code pour reduira la fourchette a chaque fois sans lancer un code qui tourne pendant 20h
param_grid = {'model__n_estimators': [550,600,650,700,800], 'model__max_depth': [2,3,4,6], 'model__learning_rate': [ 0.5,0.55,0.575, 0.6]}

if (False):
    grid_search = GridSearchCV(pipelineXGB_opti, param_grid, scoring='f1')
    grid_search.fit(OH(train, hot_cols), train_Y)


    print(grid_search.best_params_)

# Code pour le modèle avec paramètres améliorés
pipelineXGB= imbPipeline(steps = [
    ("clean", StringCleanTransformer()),
    ("amout to sign", SignTransformer()),
    ("day_time_separator", DayTimeTransformer()),
    ("Dropper", DropperTransformer(drop_cols)),
    ("One hot encoding", OHTransformer(hot_cols)),
    ("weekday", weekdayTransformer()),
    ("float", FloatTransformer()),
    ("smote", smt),
    ("model", XGBClassifier(learning_rate = 0.5, max_depth = 4, n_estimators = 600))
])

if (False):
    copy_train = train.copy()
    copy_train_Y = train_Y.copy()
    pipelineXGB.fit(copy_train, copy_train_Y)
    test_res = pipelineXGB.predict(test.copy())

    output = pd.DataFrame()
    output["TransactionId"] = test["TransactionId"]
    output["FraudResult"] = test_res

    #save the result to csv file
    output.to_csv("submission_xgb_opti.csv", index=False)


## Optimisation du pipeline

Plusieurs modifications au pipeline ont été essayées:
-   Modifier le ratio final de fraude/pas fraude => meilleur résultat avec 25% de fraudes.
-   Modification des features droppées ou One Hot encodée => plusieurs variantes ont été testée, le meilleurs résultat actuel est optenu avec : drop_cols_opti = ["CurrencyCode", "BatchId", "CountryCode", "CustomerId", "PricingStrategy", "Amount", "ProductCategory"] et hot_cols = ["ProductCategory"]
-   Ajout binning de la période de la journée à la place de l'heure => reduit les performances du modèle
-   Transformation de la date en jour de la semaine => améliore les performances du modèle
-   Ajout d'une colonne "total" qui contient la somme des transactions pour chaque client précédant la date de la transaction actuelle => Déteriore grandement les performances du modèle.


|  | Public score | Private score |
| --- | --- | --- |
| Best score (smote 0.25)| 0.754098360 | 0.749999999 |
| smote 0.5 | 0.741935483 | 0.744186046  |
| smote 0.25 binning | 0.741935483 | 0.702290076 |
| smote 0.25 sans weekDayTransformer | 0.657142857 | 0.680851063 |
| smote 0.25 avec total | 0.584615384 | 0.610687022 |


In [15]:
# Code pour le modèle final
smt = SMOTE(sampling_strategy=0.5)

drop_cols_opti = ["CurrencyCode", "BatchId", "CountryCode", "CustomerId", "PricingStrategy", "Amount", "ProductCategory"]
hot_cols = ["ProductCategory"]

pipelineXGB= imbPipeline(steps = [
    ("clean", StringCleanTransformer()),
    ("amout to sign", SignTransformer()),
    ("day_time_separator", DayTimeTransformer()),
    ("Dropper", DropperTransformer(drop_cols)),
    ("One hot encoding", OHTransformer(hot_cols)),
    ("weekday", weekdayTransformer()),
    ("float", FloatTransformer()),
    ("smote", smt),
    ("model", XGBClassifier(n_estimators = 500))
])


copy_train = train.copy()
copy_train_Y = train_Y.copy()
pipelineXGB.fit(copy_train, copy_train_Y)
test_res = pipelineXGB.predict(test.copy())

output = pd.DataFrame()
output["TransactionId"] = test["TransactionId"]
output["FraudResult"] = test_res

#save the result to csv file
output.to_csv("submission_xgb_0.25.csv", index=False)

## Conclusion et perspectives

Le meilleur score obtenu est le suivant: 

![Best Result](./bestScore.png)

Le score obtenu nous semble satisfaisant, d'autant plus qu'il nous place parmi les 128 meilleurs scores du concours, mais nous pensons qu’il est possible de l’améliorer encore. Nous avons plusieurs pistes pour y parvenir qui nous semblent intéressantes à explorer:
-   Faire du binning sur l'heure. Le binning testé n'a été testé qu’avec 7 "bin" et nous n'avons pas essayé d'associer cette feature à d'autres.
-   Modifier le calcul du total de transaction afin de calculer le total sur une période définie et/ou ajouter d'autres paramètres au calcul du total (batchId, channelId,...)
-   Testé d'autres ratios de fraudes/pas fraudes pour le SMOTE et expérimenter avec d'autre méthode d'équilibrage (entrainement de plusieurs modèles sans faire d'under/oversampling en utilisant des subset créer a partir de toutes les fraudes et autan de "pas fraudes" sélectionnés au hasard et ensuite faire la moyenne de ces modèles...)
-   Optimiser les hyperparamètres, cela nécessite cependant de résoudre le problème d'estimation de performances du modèle.
-   Faire davantage de feature engineering avec davantage de domaine knowledge. Au regard des résultats obtenus par les participants au concours, il doit y avoir une manipulation de données qui permet d'améliorer nettement le score, mais à ce stade-ci nous manquons probablement de connaissances spécifiques pour la trouver.
